# Language Knowledge (Vocabulary)
Duration: 30 minutes
Content: This section tests your knowledge of Japanese vocabulary, including kanji readings, orthography, word formation, contextually-defined expressions, paraphrases, and usage
It mainly composes following five categories:
- ``Reading Kana`` (Pronunciation Questions): Given a kanji word, choose the correct kana reading.
- `Writing Kanji` (Writing Questions): Given a word written in kana, choose the correct kanji representation.
- `Word Meaning` Selection (Vocabulary Understanding): Choose the most suitable word to fill in the sentence from four options.
- `Synonym Replacement`: Select a word that has the same or similar meaning as the underlined word.
- `Vocabulary Usage`: Assess the usage of words in actual contexts, choosing the most appropriate word usage, including some common Japanese expressions or fixed phrases.

In [10]:
import pandas as pd
import json
import os
import pickle
import re
import uuid
from typing import *
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Latex
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

load_dotenv()

True

In [11]:
# Import N3 Vocabulary
file_path = '../Vocab/n3.csv'
# Read the CSV file
data = pd.read_csv(file_path)
words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
# Display the content of the CSV file
words.head()

,expression,reading
0,映る,うつる
1,光景,こうけい
2,具体,ぐたい
3,不利,ふり
4,指揮,しき


In [12]:
os.environ["AZURE_API_KEY"]

'EEkcOWi5qXOYAPyAGPemO4DU5yuucgygVkdeSBaUUbxMP1P7CqaNJQQJ99BDACHYHv6XJ3w3AAAAACOGKTOn'

#### load Models

In [13]:
azure_llm = AzureChatOpenAI(
    azure_endpoint="https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview",
    api_key=os.environ["AZURE_API_KEY"],
    model_name="gpt-4o",
    api_version="2025-01-01-preview",
    temperature=0.5,
)

In [14]:
# aws_llm = ChatBedrock(
#     model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
#      # model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
#     model_kwargs=dict(temperature=0.3),
#     region = "us-east-2",
#     aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
#     aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
# )

## Question Explore

# Kanji 读假名（读音问题）

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class QuestionState(TypedDict):
    topic: str
    question: str
    improved_question: str
    final_question: str


kanji_example = """
## 問題 1

### 1 番 正解：3

> 会社で課長と男の人が話しています。男の人は出張レポートのことを国きなればなみませんか。
>
> 女：田中さん。初めての出張、お疲れ様でした、この出張のレポート詳みました。
> 男：はい。
> 女：出張の目的と訪問した会社で誰に会ったのかはこれています。ただ、話し合いについては最終的にどうなったのかがわかりくいています。そこを直してください。
> 男：はい、わかりました。
> 女：次の訪問日は3ヶ月後になつたんですね。
> 男：はい。
>
> 男の人は出張レポートのことを直きなければなりませんか。

### 2 番 正解：3

> 図書館で男の学生と受付の人が話しています。男の学生は本の子をずるためにこの後、何をしますか。
>
> 男：すみません。昔れたし本があるんですが、図書館のパソコンで調べたら貸し出し中になっていて、子でっていう件があきけと押せんいんです。
> 女：すみません。その本の名前は今、問題があって使えないてなっています。あの、図書館の利用カードは持っていますか。
> 男：はい。
> 女：それではうちの図書館に貸して出してしたければ予约できますよ。
> 男：あ、そうですか。わかりました。
## 会話内容

### 1番 正解：3

> 女：あ、ただ、借りているつしゃの本の中に貸し出し期限を過ぎた本があると予約できるって子的できるが…。
> 男：それは大丈夫です。ありがとうごさいます。

男の学生は本の予約をするためにこの後、何をしますか。

### 3番 正解：1

> 大学の音楽クラブの部室で女の学生と男の学生が話しています。女の学生は之後、何をしますか。
>
> 女：遅くなってごめん。明日のコンサートの準備、もう始まってると聞いた？みんなもう会場の準備してる？
> 男：あ、伊藤さん。みんな体育館に椅子を並べに行ったよ。伊藤さんも行ってくれる？
> 女：今、ちょっとプログラム印刷し終わったから持って、体育館の入口で受け用的テーブルがあるからその上に置いて。
> 男：わかりました。
> 女：わかって。
> 男：その後、みんなと一緒に椅子、並べくれる？
> 女：OK。楽器を運ぶのはその後？大家い内的に今日のうちに運びだせな。
> 男：ああ、さっき体育館に行った時に最初に運んでもらった。僕は先生に明日のこと相談して行ったから体育館に向かうね。
> 女：わかって。

女の学生はこの後、何をしますか。

### 4番 正解：2

> 会社で女の人と男の人が話しています。女の人は之後まず何しますか。
>
> 女：村上さん。今年の新入社員のセミナー、来月ですが、1日目的予定表はこれでよろしいですか？
> 男：ああ、はい。えっと、9時スタートで社長の話。その後、昼までビジネスマナーの先生の講義、去年と同じだね。あー、毎年9時スタートで朝から準備で忙しいという意見が多くて。
> 女：そうでしたか。
> 男：それで今年は30分遅くして9時半開始にしようという話になったんだ。終わる


のは１２時じゃなくて１２時半になるけど。  
> 女：はい。  
> 男：会場は一応、午後１時まで使ってるから問題ないよ。社員に去年より３０分遅くなってもいいか、言合せ聞いてなくて、いちいちのは最初のところだけだから大丈夫だと思うけど。  
> 女：わかりました。  
> 男：先生には今日会うことになってるから確認しておくよ。全部確認取れてから予定表、直してくれる？  
> 女：はい。

質問：
**女の人はこの後まず何をしますか。**  
**正解：** 4

---

（５番）：

> 健康診断で医者と男の人が話しています。男の人はこれから何を変えますか。

女（医者）： 検査結果は特に問題はなさそうですね。健康のことで何か気になることはありますか？  
男： 最近、夜、なかなか眠れないことが多くて困ってるんです。ストレスは特にないんですけど・・・  
>女 *寝る直前まで携帯を見たり、寝る時間が不規則だったりすると眠れなくなることがありますよ。*

**男：** 布団に入る時間はいつも１１時頃で、携帯は寝る前に見ないようにしています。  
>女 *そうですか。それとも食事してすぐ寝るのもよくないですよ。*

**男：** 仕事が忙しい日は時々遅くなります。  
>女 *３時間前に食事をしても大丈夫ですか？夕食の内容はどうですか？ たとえば、フライとか油をたくさん使ったものが多いようでしたら和食にして、今度は軽いものにしてみてください。*

**男：** 好物なので、つい脂っこいというのも多いですね。でもやってみます。  
>女 *年齢に合わせて、生活習慣に気をつけてくださいね。*

#### 質問：
**男の人はこれから何を変えますか。**  
**正解：** 4

"""

print(words)

# Nodes
def generate_question(state: QuestionState):
    """First LLM call to generate initial question"""

    msg = azure_llm.invoke(f"You are a japanese teacher. Your job is to write a paper for Preparing the original text and options for the listening dialogue based on the reference format. Students need to listen to dialogues and choose options that match the meaning of the question based on the listening content. Some questions have picture options, while others are mostly text options. There will be 3-5 back and forth dialogues with around 200-300 words. The roll only displays options. Listen often after the conversation, ask one person in the conversation what they want to do next. Only refer to the format, not the content."
            f"The vocabulary should be restricted to N3 level, you can refer to the vocabulary in the word list, choosing random words for the questions"
            f"please refer the question examples following the formal exam paper"
            f"append the correct answer and explanation of main challenges and why teacher asks this question to candidate in chinese at each question"
            f"Finally, beautify markdown format"
            f"topic list: {state['topic']}"
            f"word list: {words}"
            f"formal exam paper: {kanji_example}")
    
    return {"question": msg.content}


# Build workflow
kanji_workflow = StateGraph(QuestionState)

# Add nodes
kanji_workflow.add_node("generate_question", generate_question)

# Add edges to connect nodes
kanji_workflow.add_edge(START, "generate_question")
kanji_workflow.add_edge("generate_question", END)

# Compile
kanji_graph = kanji_workflow.compile()


     expression reading
0            映る     うつる
1            光景    こうけい
2            具体     ぐたい
3            不利      ふり
4            指揮      しき
...         ...     ...
2134          優      ゆう
2135         最低    さいてい
2136         救助   きゅうじょ
2137        煮える     にえる
2138         就く      つく

[2139 rows x 2 columns]


In [16]:
# Show workflow
#display(Image(kanji_graph.get_graph().draw_png()))

In [17]:
# Invoke
kanji = kanji_graph.invoke({"topic": "店で価格を尋ねる | レストランで食べ物を注文する | 交通手段について話す| 天気の状況について話す | 趣味について話す | 店で商品の特徴や価格を尋ねる | スーパーマーケットで買い物をする | レストランで食べ物や飲み物を注文する | カフェでテイクアウトを頼む | コンビニで支払い方法について質問する |  ネットショッピングで商品を選ぶ"})
display(Markdown(kanji["question"]))

以下是基于您的要求编写的正式格式的试题，包括四篇短文、问题及答案选择，附带正确答案和解释，并用Markdown格式美化。

---

## 問題 4

つぎの(1)から(4)の文章を読んで、質問に答えなさい。答えは、1・2・3・4から最もよいものを一つえらびなさい。

---

### (1)

これは、ネットショッピングで商品を選ぶときのメールである。

---

**メール**

こんにちは、田中さん。

この前、教えてくれたサイトでバッグを見ていましたが、どれがいいか迷っています。黒いバッグと青いバッグ、どちらが仕事に合うと思いますか？黒い方がフォーマルに見えますが、青い方は少し個性的で、私の服に合いそうです。

それから、どちらも価格は10,000円ですが、黒いバッグの方が軽くて持ちやすいと書いてありました。アドバイスをお願いします。

よろしくお願いします。

佐藤

---

**質問**

1. 佐藤さんはどちらのバッグを選ぶべきですか？
   1. 青いバッグ
   2. 黒いバッグ
   3. どちらも買わない
   4. 価格が高いバッグ

**正解**：2. 黒いバッグ  
**解説**：黒いバッグはフォーマルで、仕事に合うと書かれており、さらに軽くて持ちやすいという具体的な利点が記述されています。  
**挑戦点**：この问题考察了考生对细节的理解能力，要求他们从多个描述中选择出最符合逻辑的答案。  
**为什么问这个问题**：通过此问题，教师可以评估考生是否能够从文本中提取关键信息，尤其是在对比描述的情况下。  
**中文解释**：通过这道题目，考察了学生对细节描述的理解和逻辑判断能力。

---

### (2)

これは、カフェでテイクアウトを頼むときの会話である。

---

**会話**

店員: いらっしゃいませ。ご注文はお決まりですか？  
客: はい、ホットコーヒーを1つと、チーズケーキを1つお願いします。  
店員: かしこまりました。お持ち帰りですか？  
客: はい、テイクアウトでお願いします。  
店員: では、ホットコーヒーはMサイズでよろしいですか？  
客: はい、それでお願いします。  

---

**質問**

2. 客はどのように注文しましたか？
   1. 店内でコーヒーを飲む
   2. ホットコーヒーとケーキをテイクアウトする
   3. コーヒーだけをテイクアウトする
   4. ケーキだけを店内で食べる

**正解**：2. ホットコーヒーとケーキをテイクアウトする  
**解説**：客は「ホットコーヒーを1つと、チーズケーキを1つ」と注文し，并明确表示“テイクアウトでお願いします”。  
**挑戦点**：需要学生注意对话中的关键词汇，如“テイクアウト”以及具体的食物点单内容。  
**为什么问这个问题**：教师通过此问题评估学生对实际生活场景中常见表达的理解能力，尤其是在点单场景下的交流能力。  
**中文解释**：此题目旨在评估学生对实际生活中常见对话的理解能力。

---

### (3)

これは、店で商品の特徴や価格を尋ねるときの会話である。

---

**会話**

客: すみません、このシャツはいくらですか？  
店員: こちらは3,500円です。今、セール中で、2枚買うと1枚は半額になります。  
客: そうですか。素材は何ですか？  
店員: 綿100%ですので、肌触りがいいですよ。  
客: ありがとうございます。少し考えます。

---

**質問**

3. このシャツの特徴は何ですか？
   1. 綿100%で、肌触りがいい
   2. セール中で、2枚買うと1枚無料
   3. 価格が高い
   4. サイズが豊富

**正解**：1. 綿100%で、肌触りがいい  
**解説**：会話中明确提到“綿100%ですので、肌触りがいいですよ”，这是商品的主要特点。  
**挑戦点**：需要考生从对话中提取商品的具体特点，同时排除干扰选项。  
**为什么问这个问题**：教师通过此问题评估学生在购物场景下的听力和阅读理解能力。  
**中文解释**：通过这道题目，考察学生在购物场景中对商品描述的理解能力。

---

### (4)

これは、天気の状況について話すときのニュースである。

---

**ニュース**

今日の午後は、全国的に雨が降りやすい天気になります。特に西日本では、雷を伴う激しい雨になる可能性がありますので、注意してください。一方、東日本では、夜になると雨が止むでしょう。

---

**質問**

4. 西日本の午後の天気はどうなる可能性がありますか？
   1. 雨が止む  
   2. 雷を伴う激しい雨が降る  
   3. 晴れる  
   4. 雪が降る  

**正解**：2. 雷を伴う激しい雨が降る  
**解説**：ニュース明确提到“西日本では、雷を伴う激しい雨になる可能性があります”。  
**挑戦点**：需要学生关注天气预报中的细节描述，并选择最准确的答案。  
**为什么问这个问题**：教师通过此问题评估学生对天气预报场景的理解能力，尤其是对预测性语言的掌握。  
**中文解释**：此题目旨在考察学生对天气预报场景中细节信息的理解能力。

---

以上问题严格控制在N3词汇范围内，涵盖了实际生活中常见的场景，如购物、点单、天气等，帮助学生在考试中更好地适应实际语言应用场景。